In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

# Data Preprocessing - clearinghouse data

In [ ]:
rankings = pd.read_excel("2022BC-national-universities.xlsx")
ch13_18 = pd.read_excel("Clearinghouse2013to2018.xlsx", dtype=str)
ch_20 = pd.read_excel("CH_20.xlsx", dtype=str)
ch_21 = pd.read_excel("CH_21.xlsx", dtype=str)

In [ ]:
ch_20["Ref ID"] = ch_20.Field.str.split(pat=".", expand=True)[0]
ch_20 = ch_20.drop(columns=["Field"])
ch_21["Ref ID"] = ch_21.Field.str.split(pat=".", expand=True)[0]
ch_21 = ch_21.drop(columns=["Field"])

In [ ]:
ch_20 = ch_20.drop_duplicates(subset='Ref ID', keep="first")
ch_21 = ch_21.drop_duplicates(subset='Ref ID', keep="first")
ch13_18 = ch13_18.drop_duplicates(subset='Ref ID', keep="first")
ch_20 = ch_20[ch_20["College Name"].notna()]
ch_21 = ch_21[ch_21["College Name"].notna()]
ch13_18 = ch13_18[ch13_18["College Name"].notna()]
ch_20 = ch_20[ch_20["Ref ID"].notna()]
ch_21 = ch_21[ch_21["Ref ID"].notna()]
ch13_18 = ch13_18[ch13_18["Ref ID"].notna()]

In [ ]:
drop_columns=["Enrollment Begin", "Enrollment End", "Enrollment Status", 
              "Record Found Y/N", "Class Level", "Graduated?", "Graduation Date", 
              "2-year / 4-year", "Public / Private", "College Sequence", 
              "College Code/Branch"]
ch_20 = ch_20.drop(columns=drop_columns)
ch_20["Period"] = ch_20["Period"].astype(float).astype(int)
ch_21 = ch_21.drop(columns=drop_columns)
ch_21["Period"] = ch_21["Period"].astype(float).astype(int)
ch13_18 = ch13_18.drop(columns=drop_columns)
ch13_18["Period"] = ch13_18["Period"].astype(float).astype(int)
ch13_18["Ref ID"] = ch13_18["Ref ID"].astype(float).astype(int)

In [ ]:
clearing_house = pd.concat([ch13_18, ch_20, ch_21], axis=0)

In [ ]:
ch_replace = {
  "UNDERGADUATE": "",
  "GRADUATE": "",
  "- GRADS" : "",
  "- GRAD SCHOOL": "",
  "GRAD": "",
  "COLLEGE OF MEDICINE-AECOM-GRAD": "",
  "COLLEGE OF MEDICINE": "",
  "- FRESNO": "",
  "-DOCTORAL": "",
  "-ORANGE": "",
  "- PUEBLO": "",
  "- BROOKLYN": "",
  "- INTEGRATED HEALTH SCIENCES": "",
  "HEALTH SCIENCES CENTER": "", 
  "- HEALTH SCIENCES": "",
  "- HEALTH": "",
  "- KUNSHAN": "",
  "- LAW SCHOOL": "", 
  "-UNDERGRADS": "", 
  "-TRADITIONAL": "",
  "- BOZEMAN": "",
  "- MAIN": "",
  "- STILLWATER/TULSA": "",
  " - PLEASANTVILLE": "",
  "TEACHERS COLLEGE": "",
  "IN HUNTSVILLE": "",
  "- COLORADO": "",
  "- (NON-MED)": "",
  " - LAW/SPCS": "", 
  "-BRANCH 81": "",
  "-DURAM": "",
  "-SEMESTERS": "",
  "- DEGREE SEEKING": "",
  "- JESUIT SCHOOL OF THEOLOGY": "",
  "-DARTMOUTH": " - DARTMOUTH",
  "-RENO": " - RENO",
  " DENVER": "DENVER",
  "-TWIN CITIES": " - TWIN CITIES",
  "-DAVIS": " - DAVIS",
  "-COLLEGE PARK": " - COLLEGE PARK",
  "-CHAPEL HILL": " - CHAPEL HILL",
  "-GREENSBORO": " - GREENSBORO",
  "-WILMINGTON": " - WILMINGTON",
  "-LOS ANGELES": " - LOS ANGELES",
  "-BERKLEY": " - BERKLEY",
  "-SAN FRANCISCO": " - SAN FRANCISCO",
  "-SANTA CRUZ": " - SANTA CRUZ",
  "-SAN DIEGO": " - SAN DIEGO",
  "-SANTA BARBARA": " - SANTA BARBARA",
  "BAYLOR COLLEGE OF MEDICINE": "BAYLOR UNIVERSITY",
  "PENN STATE-HERSHEY PARK MED CTR": "PENNSYLVANIA STATE UNIVERSITY",
  "ST JOHN FISHER COLLEGE": "ST JOHN FISHER COLLEGE"
}

ch_search_replace = {
    "HARVARD": "HARVARD UNIVERSITY",
    "JOHNS HOPKINS": "JOHNS HOPKINS UNIVERSITY",
    "LOUISIANA STATE": "LOUISIANA STATE UNIVERSITY",
    "PURDUE": "PURDUE UNIVERSITY",
    "STONY BROOK": "STONY BROOK UNIVERSITY",
    "BINGHAMTON": "BINGHAMTON UNIVERSITY",
    "BUFFALO" : "UNIVERSITY AT BUFFALO",
    "ALBANY": "UNIVERSITY AT ALBANY",
    "TEXAS A&M": "TEXAS A&M UNIVERSITY",
    "OHIO STATE": "OHIO STATE UNIVERSITY",
    "UNIVERSITY OF ALABAMA": "UNIVERSITY OF ALABAMA",
    "UNIVERSITY OF ARKANSAS": "UNIVERSITY OF ARKANSAS",
    "UNIVERSITY OF MISSOURI": "UNIVERSITY OF MISSOURI",
    "UNIVERSITY OF ROCHESTER": "UNIVERSITY OF ROCHESTER",
    "UNIVERSITY OF TENNESSEE": "UNIVERSITY OF TENNESSEE",
    "WASHINGTON UNIVERSITY": "WASHINGTON UNIVERSITY IN ST. LOUIS",
    "WAKE FOREST UNIVERSITY": "WAKE FOREST UNIVERSITY",
    "UNIVERSITY OF MICHIGAN": "UNIVERSITY OF MICHIGAN - ANN ARBOR",
    "YALE": "YALE UNIVERSITY",
    "BAYLOR": "BAYLOR UNIVERSITY",
    "RUTGERS": "RUTGERS UNIVERSITY",
    "DUKE" : "DUKE UNIVERSITY",
    "SOUTHERN NEW HAMPSHIRE": "SOUTHERN NEW HAMPSHIRE UNIVERSITY"
}

rank_replace = {
    "--Oxford":  "",
    "--St.Louis": "",
    "--West Lafayette": "",
    "--New Brunswick": "",
    "--Columbus": "",
    "--Newark": "",
    "Univ. ": "UNIVERSITY ",
    "U. ": "UNIVERSITY ",
    "Inst. ": "INSTITUTE ",
    "CUNY--City College": "CUNY CITY COLLEGE",
    "Indiana University--Bloomington": 'INDIANA UNIVERSITY BLOOMINGTON',
    "Louisiana State University--Baton Rouge": "LOUISIANA STATE UNIVERSITY",
    "Stony Brook--SUNY" : "STONY BROOK UNIVERSITY",
    "University at Albany--SUNY": "UNIVERSITY AT ALBANY",
    "University at Buffalo--SUNY": "UNIVERSITY AT BUFFALO",
    "Binghamton University--SUNY": "BINGHAMTON UNIVERSITY",
    "--Manoa": " AT MANOA",
    "--Hilo": " AT HILO",
    "--Chicago": " AT CHICAGO",
    "--Amherst": " AT AMHERST",
    "--Omaha": " AT OMAHA",
    "--Boston": " BOSTON",
    "--Lowell": " LOWELL",
    "--Urbana-Champaign": " @ URBANA",
    "--Las Vegas": " LAS VEGAS",
    "--": " - "
}

In [ ]:
rankings["Institution"] = rankings["Institution"].str.replace(r"\(.*\)","")
for key, value in rank_replace.items():
  rankings["Institution"] = rankings["Institution"].str.replace(key,value)
rankings["Institution"] = rankings["Institution"].str.upper()
rankings["Institution"] = rankings["Institution"].str.strip()

for key, value in ch_search_replace.items():
  clearing_house = clearing_house.applymap(lambda x: value if key in str(x) else x)

for key, value in ch_replace.items(): 
  clearing_house["College Name"] = clearing_house["College Name"].str.replace(key, value)
clearing_house["College Name"] = clearing_house["College Name"].str.strip()
clearing_house.loc[(clearing_house["College Name"] == "WASHINGTON UNIVERSITY IN ST. LOUIS") & (clearing_house["College State"] == "DC"), "College Name"] = "GEORGE WASHINGTON UNIVERSITY"

rank, college_list = [], rankings['Institution'].tolist()
remain = []
for college in clearing_house["College Name"]:
  if college in college_list:
    rank.append(rankings.loc[rankings['Institution'] == college, '2022 Rank'].values[0])
  else:
    rank.append(300)
clearing_house["Rank Enrolled"] = rank
clearing_house["Rank Difference"] = clearing_house["Rank Enrolled"] - 34
clearing_house["Search Date"] = clearing_house["Search Date"].apply(lambda x: datetime.date(int(x[:4]), int(x[4:6]), int(x[6:8])))
clearing_house = clearing_house.set_index('Ref ID')

In [ ]:
clearing_house.to_excel("CH_CLEAN.xlsx")

# Data Preprocessing - application data

In [ ]:
admissions = pd.read_excel("Capstone Dataset 20210608-183000.xlsx")
admissions = admissions.drop_duplicates(subset='Ref ID', keep="first")
admissions = admissions[admissions["Decision 1"].notna()]

In [ ]:
admissions["Decision 1"] = admissions["Decision 1"].apply(lambda x: str(x).ljust(len(str(x))+1))
admissions["Decision 2"] = admissions["Decision 2"].apply(lambda x: str(x).ljust(len(str(x))+1))
admissions["Decision 3"] = admissions["Decision 3"].apply(lambda x: str(x).ljust(len(str(x))+1))
decision_concat = admissions.loc[:, ["Decision 1","Decision 2", "Decision 3"]].fillna('').sum(axis=1)
perms = sorted(list(set(decision_concat)), key=len)
admissions["Decision Type"] = decision_concat.apply(lambda x: perms.index(x) + 1)
perms
                                                    

In [ ]:
admissions['Institution 1 Name'] = admissions['Institution 1 Name'].fillna('').str.upper()
admissions['Institution 2 Name'] = admissions['Institution 2 Name'].fillna('').str.upper()
admissions['Institution 3 Name'] = admissions['Institution 3 Name'].fillna('').str.upper()
admissions['Institution 4 Name'] = admissions['Institution 4 Name'].fillna('').str.upper()

for key, value in ch_search_replace.items():
  admissions = admissions.applymap(lambda x: value if key in str(x) else x)

for key, value in ch_replace.items(): 
  admissions['Institution 1 Name'] = admissions['Institution 1 Name'].str.replace(key, value)
  admissions['Institution 2 Name'] = admissions['Institution 2 Name'].str.replace(key, value)
  admissions['Institution 3 Name'] = admissions['Institution 3 Name'].str.replace(key, value)
  admissions['Institution 4 Name'] = admissions['Institution 4 Name'].str.replace(key, value)

admissions['Institution 1 Name'] = admissions['Institution 1 Name'].str.strip()
admissions['Institution 2 Name'] = admissions['Institution 2 Name'].str.strip()
admissions['Institution 3 Name'] = admissions['Institution 3 Name'].str.strip()
admissions['Institution 4 Name'] = admissions['Institution 4 Name'].str.strip()

cats = ['Institution 1 Name', 'Institution 2 Name', 'Institution 3 Name', 'Institution 4 Name']
college_list = rankings['Institution'].tolist()
for col in cats:
  rank = []
  for college in admissions[col]:
    if college in college_list:
      rank.append(rankings.loc[rankings['Institution'] == college, '2022 Rank'].values[0])
    elif college == "":
      rank.append("")
    else:
      rank.append("Not Available")
  admissions[col.replace("Name", "Rank")] = rank
admissions = admissions.set_index("Ref ID")

In [ ]:
admissions.to_excel("Admissions_CLEAN.xlsx")

# Statistical Tests

In [ ]:
dec_type = pd.read_excel("Decision_Type.xlsx", index_col=[0])
dec_type = dec_type.rename(columns={"Decision 1": "D1", "Decision 2": "D2", "Decision 3": "D3", "UR Decision": "URD", "Applicant Decision": "AD"})
dec_type.index = dec_type.index.rename("Index")
dec_type["URDAD"] = dec_type["URD"] + dec_type["AD"] 
dec_int = list(set(dec_type["URD"] + dec_type["AD"] ))
dec_type["Decision Final"] = dec_type["URDAD"].apply(lambda x: dec_int.index(x) + 1)
admissions["Decision Final"] = admissions["Decision Type"].apply(lambda x: dec_type[dec_type.index == x]["Decision Final"].iloc[0])
dec_type

In [ ]:
merged_df = admissions.merge(clearing_house, how="inner", on="Ref ID")
merged_df.columns

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(8.7,8.27)})
sns.set(style="darkgrid")
fig, ((ax1), (ax3)) = plt.subplots(2,1)
sns.boxplot(merged_df[merged_df["Decision Final"]==5]["Rank Difference"], ax=ax1, palette="Set2")
ax1.set_title("RANK DIFFERENCE OF THOSE DENIED BY UR")

sns.boxplot(merged_df[merged_df["Decision Final"]==6]["Rank Difference"], ax=ax3, palette="Set2")
ax3.set_title("RANK DIFFERENCE OF THOSE WHO DIDNT ACCEPT")
fig.tight_layout()
fig.show()

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(15.7,8.27)})
sns.set(style="darkgrid")
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)
sns.boxplot(merged_df[merged_df["Decision Final"]==5]["Rank Difference"], ax=ax1, palette="Set2")
ax1.set_title("RANK DIFFERENCE OF THOSE DENIED BY UR")
sns.barplot(merged_df[merged_df["Decision Final"]==5]["College Name"].value_counts().iloc[:10], merged_df[merged_df["Decision Final"]==5]["College Name"].value_counts().iloc[:10].index, ax=ax2)
ax2.set_title("MOST COMMON ENROLLED OF THOSE DENIED BY UR")

sns.boxplot(merged_df[merged_df["Decision Final"]==6]["Rank Difference"], ax=ax3, palette="Set2")
ax3.set_title("RANK DIFFERENCE OF THOSE WHO DIDNT ACCEPT")
sns.barplot(merged_df[merged_df["Decision Final"]==6]["College Name"].value_counts().iloc[:10],merged_df[merged_df["Decision Final"]==6]["College Name"].value_counts().iloc[:10].index, ax=ax4)
ax4.set_title("MOST COMMON ENROLLED OF THOSE WHO DIDNT ACCEPT")
fig.tight_layout()
fig.show()

In [ ]:
merged_df[merged_df["Decision Final"]==5]["Rank Difference"].median()

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(8.7,8.27)})
sns.set(style="darkgrid")
fig, ((ax2), (ax4)) = plt.subplots(2,1)
merged_df["GPA AVG"] = merged_df.loc[:, ["Institution 1 GPA (4.0 Scale)","Institution 2 GPA (4.0 Scale)","Institution 3 GPA (4.0 Scale)","Institution 4 GPA (4.0 Scale)"]].mean(axis=1,skipna=True)
sns.boxplot(merged_df[merged_df["Decision Final"]==5]["GPA AVG"], ax=ax2, palette="Set1")
ax2.set_title("GPA OF THOSE DENIED BY UR")
ax2.set_xlim(2.25,4)

sns.boxplot(merged_df[merged_df["Decision Final"]==6]["GPA AVG"], ax=ax4, palette="Set1")
ax4.set_title("GPA OF THOSE WHO DIDN'T ACCEPT")
ax4.set_xlim(2.25,4)
fig.tight_layout()
fig.show()

In [ ]:
merged_df[merged_df["Decision Final"]==3]["GPA AVG"]

In [ ]:
from scipy.stats import ttest_ind
ttest_ind(merged_df[merged_df["Decision Final"]==3]["GRE Quantitative Percentile"].dropna(), merged_df[merged_df["Decision Final"]==6]["GRE Quantitative Percentile"].dropna())

In [ ]:
merged_df.loc[:, ["Institution 1 Name","Institution 2 Name","Institution 3 Name","Institution 4 Name"]][merged_df["Decision Final"]==5].stack().value_counts().iloc[:10]

In [ ]:
sns.set(rc={'figure.figsize':(15.7,8.27)})
sns.set(style="darkgrid")
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)
merged_df["Rank AVG"] = merged_df.loc[:, ["Institution 1 Rank","Institution 2 Rank","Institution 3 Rank","Institution 4 Rank"]].replace("Not Available", np.nan).mean(axis=1,skipna=True)
sns.boxplot(merged_df[merged_df["Decision Final"]==5]["Rank AVG"], ax=ax1, palette="Set2")
ax1.set_title("PREVIOUS RANK OF THOSE DENIED BY UR")
ax1.set_xlim(0,175)
sns.barplot(merged_df.loc[:, ["Institution 1 Name","Institution 2 Name","Institution 3 Name","Institution 4 Name"]][merged_df["Decision Final"]==5].stack().value_counts().iloc[1:11], merged_df.loc[:, ["Institution 1 Name","Institution 2 Name","Institution 3 Name","Institution 4 Name"]][merged_df["Decision Final"]==5].stack().value_counts().iloc[1:11].index, ax=ax2)
ax2.set_title("PREVIOUS SCHOOLS OF THOSE DENIED BY UR")

sns.boxplot(merged_df[merged_df["Decision Final"]==6]["Rank AVG"], ax=ax3, palette="Set2")
ax3.set_title("PREVIOUS RANK OF THOSE WHO DIDNT ACCEPT")
ax3.set_xlim(0,175)
sns.barplot(merged_df.loc[:, ["Institution 1 Name","Institution 2 Name","Institution 3 Name","Institution 4 Name"]][merged_df["Decision Final"]==6].stack().value_counts().iloc[1:11], merged_df.loc[:, ["Institution 1 Name","Institution 2 Name","Institution 3 Name","Institution 4 Name"]][merged_df["Decision Final"]==6].stack().value_counts().iloc[1:11].index, ax=ax4)
ax4.set_title("PREVIOUS SCHOOLS OF THOSE WHO DIDN'T ACCEPT UR")
fig.tight_layout()
fig.show()

In [ ]:
ttest_ind(merged_df[merged_df["Decision Final"]==6]["GRE Quantitative Percentile"].dropna(), merged_df[merged_df["Decision Final"]==1]["GRE Quantitative Percentile"].dropna())

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set(style="darkgrid")
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3,2)
sns.boxplot(merged_df[merged_df["Decision Final"]==5]["GPA AVG"], ax=ax1, palette="Set1")
ax1.set_title("GPA OF THOSE DENIED BY UR")
ax1.set_xlim(2.7,4)
sns.boxplot(merged_df[merged_df["Decision Final"]==5]["GRE Quantitative Percentile"], ax=ax2, palette="Set2")
ax2.set_title("GRE OF THOSE DENIED BY UR")
ax2.set_xlim(80,100)

sns.boxplot(merged_df[merged_df["Decision Final"]==1]["GPA AVG"], ax=ax3, palette="Set1")
ax3.set_title("GPA OF THOSE WHO ACCEPTED")
ax3.set_xlim(2.7,4)
sns.boxplot(merged_df[merged_df["Decision Final"]==1]["GRE Quantitative Percentile"], ax=ax4, palette="Set2")
ax4.set_title("GRE OF THOSE WHO ACCEPTED")
ax4.set_xlim(80,100)

sns.boxplot(merged_df[merged_df["Decision Final"]==6]["GPA AVG"], ax=ax5, palette="Set1")
ax5.set_title("GPA OF THOSE WHO DIDN'T ACCEPT")
ax5.set_xlim(2.7,4)
sns.boxplot(merged_df[merged_df["Decision Final"]==6]["GRE Quantitative Percentile"], ax=ax6, palette="Set2")
ax6.set_title("GRE OF THOSE WHO DIDN'T ACCEPT")
ax6.set_xlim(80,100)
fig.tight_layout()
fig.show()

In [ ]:
merged_df[merged_df["Decision Final"]==1]["GPA AVG"]

In [ ]:
import numpy as np
import matplotlib.pylab as pl
import matplotlib.gridspec as gridspec
sns.set(rc={'figure.figsize':(12.7,8.27)})
sns.set(style="darkgrid")
# Create 2x2 sub plots
gs = gridspec.GridSpec(2, 2)
pl.figure()

ax = pl.subplot(gs[0]) # row 0, col 0
ax.set_title("GPA OF THOSE WHO ACCEPTED")
ax.set_xlim(3,4)
sns.boxplot(merged_df[merged_df["Decision Final"]==2]["GPA AVG"], ax=ax, palette="Set3")

ax = pl.subplot(gs[0, 1]) # row 0, col 1
ax.set_title("GRE OF THOSE WHO ACCEPTED")
ax.set_xlim(80,100)
sns.boxplot(merged_df[merged_df["Decision Final"]==2]["GRE Quantitative Percentile"], ax=ax, palette="Set1")

ax = pl.subplot(gs[1, :]) # row 1, span all columns
ax.set_title("PREVIOUS RANK OF THOSE WHO ACCEPTED")
ax.set_xlim(0,50)
sns.boxplot(merged_df[merged_df["Decision Final"]==2]["Rank AVG"], ax=ax, palette="Set1")

# fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)
# sns.boxplot(merged_df[merged_df["Decision Final"]==6]["Rank AVG"], ax=ax1, palette="Set2")
# ax1.set_title("PREVIOUS RANK OF THOSE WHO ACCEPTED")
# ax1.set_xlim(0,175)
# sns.boxplot(merged_df[merged_df["Decision Final"]==6]["GRE Quantitative Percentile"], ax=ax2, palette="Set1")
# ax2.set_title("GRE OF THOSE DENIED BY UR")
# ax2.set_xlim(60,100)

# sns.boxplot(merged_df[merged_df["Decision Final"]==5]["Rank AVG"], ax=ax3, palette="Set2")
# ax3.set_title("PREVIOUS RANK OF THOSE WHO DIDNT ACCEPT")
# ax3.set_xlim(0,175)
# sns.boxplot(merged_df[merged_df["Decision Final"]==5]["GRE Quantitative Percentile"], ax=ax4, palette="Set1")
# ax4.set_title("GRE OF THOSE WHO DIDN'T ACCEPT")
# ax4.set_xlim(60,100)
# fig.tight_layout()
# fig.show()

In [ ]:
merged_df[merged_df["Decision Final"]==2]

In [ ]:
# calculate a 5-number summary
from numpy import percentile
quartiles = percentile(merged_df["Rank Difference"], [25, 50, 75])
# calculate min/max
data_min, data_max = merged_df["Rank Difference"].min(), merged_df["Rank Difference"].max()
# print 5-number summary
print('Min: %.3f' % data_min)
print('Q1: %.3f' % quartiles[0])
print('Median: %.3f' % quartiles[1])
print('Q3: %.3f' % quartiles[2])
print('Max: %.3f' % data_max)

In [ ]:
sns.histplot(merged_df['Decision Final'], bins=len(dec_int), )

In [ ]:
pd.DataFrame(perms, index=range(1, 24), columns=["Decision Type"]).to_excel("/content/drive/MyDrive/Colab Notebooks/DSC 483: Data Science Capstone/Graduate Admissions/DecisionType.xlsx")

In [ ]:
pd.DataFrame(perms, index=range(1, 24), columns=["Decision Type"])